# Pyspark Basics

- [DataBricks docs](https://docs.databricks.com/aws/en/pyspark/basics)

**ใน part นี้ คำอธิบายและ code จะอยู่ในไฟล์นี้ไฟล์เดียว**

ส่วนผลลัพธ์เบื้องต้นจากการ run command ต่างๆจะอยู่ใน [here](https://docs.google.com/document/d/1btW2acGv49PdjaKKqNCPFIvW90bwW3yyXxhhPToOLDA/edit?usp=sharing)

---

## Import data types

PySpark operations จำนวนมากกำหนดใหเราใช้ SQL functions หรือไม่ก็ต้องใช้วิธีการเดียวกันกับ Spark แบบดั้งเดิม เพราะฉะนั้นเราจะ import module ที่น่าจะจำเป็นต่อการใช้งานมาไว้ก่อน (อย่างเหมาะสม ไม่ต้อง import เยอะเกินความจำเป็น)

In [0]:
# import select functions and types
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import floor, round

# import modules using an alias
import pyspark.sql.types as T
import pyspark.sql.functions as F

---

## Create a DataFrame

มีหลายวิธีในการสร้าง DataFrame ดังนี้

### Create a DataFrame with specified values

In [0]:
# แบบตั้งต้น พื้นฐาน
df_children = spark.createDataFrame(
  data = [("Mikhail", 15), ("Zaky", 13), ("Zoya", 8)],
  schema = ['name', 'age'])
display(df_children)

In [0]:
# (อันนี้แบบระบุ Data Type 👇)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

df_children_with_schema = spark.createDataFrame(
  data = [("Mikhail", 15), ("Zaky", 13), ("Zoya", 8)],
  schema = StructType([
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True)
  ])
)
display(df_children_with_schema)

### Create a DataFrame from a table in Unity Catalog

In [0]:
# ดึงมาจากตารางใน Catalog
df_customer = spark.table('samples.tpch.customer')
display(df_customer)

### Create a DataFrame from an uploaded file

ดึงมาจาก file ใน Volume (ในที่นี้ใช้ไฟล์จาก Volume เดิมที่เราเคยทำในไฟล์ก่อนหน้านี้ไปแล้ว หรือจะลอง upload file เข้าไปใน volume อื่นๆแล้วลองดึงมาดูก็ได้)

In [0]:
# Assign this variable your full volume file path
volume_file_path = "/Volumes/workspace/default/tutorial/rows.csv"

df_csv = (spark.read
  .format("csv")
  .option("header", True)
  .option("inferSchema", True)
  .load(volume_file_path)
)
display(df_csv)

### Create a DataFrame from a JSON response


ตัวอย่างนี้ เราจะใช้ไฟล์จากลิงก์นี้ [example](https://dummyjson.com/products)

In [0]:
import requests

# Download data from URL
url = "https://dummyjson.com/products"
response = requests.get(url)

In [0]:
# focus แค่ส่วนของ products พอ
products = response.json()["products"]

# ในข้อมูล จะมีส่วนที่ Spark ไม่สามารถจัดได้ว่าจะให้อยู่ใน Data Type แบบไหน เพราะบางที่ในส่วนนั้นข้อมูลยาวบ้าง สั้นบ้าง เราเลยจัดการข้อมูลตรงนี้ก่อน
for p in products:
    p["price"] = float(p["price"])
    p["rating"] = float(p["rating"])
    p["discountPercentage"] = float(p["discountPercentage"])

# สร้าง DataFrame จาก JSON ดังกล่าว
df_json = spark.createDataFrame(products)
display(df_json)


#### Select a JSON field or object 

In [0]:
# เลือก field มาดู เช่น เราจะดู reviews ของสินค้า
display(df_json.select(df_json["reviews"]))

In [0]:
# เลือก reviewerName ที่อยู่ใน index ที่ 1 ของรีวิวทั้งหมด ในสินค้าแต่ละชิ้น
display(df_json.select(
        df_json["reviews"][1]["reviewerName"],
        df_json["reviews"][1]["rating"],
        df_json["reviews"][1]["comment"]
    ))

### Create a DataFrame from a file

[Databricks Utilties file system commands](https://docs.databricks.com/aws/en/dev-tools/databricks-utils)

In [0]:
# ดูว่าใน DataBricks มี Sample data อะไรให้บ้าง

display(dbutils.fs.ls('/databricks-datasets'))

หรือจะใช้วิธีอีกแบบคือ [Databricks CLI file system commands](https://docs.databricks.com/aws/en/dev-tools/cli/reference/fs-commands)

In [0]:
%fs ls '/databricks-datasets'

ลองดึงข้อมูลจาก Sample Dataset ที่มีใน DataBricks มาดู

In [0]:
df_population = (spark.read
  .format("csv")
  .option("header", True)
  .option("inferSchema", True)
  .load("/databricks-datasets/samples/population-vs-price/data_geo.csv")
)
display(df_population)

## Transform data with DataFrames

เราสามารถแปลงข้อมูลใน DataFrame ได้ด้วย built-in methods ไม่ว่าจะเป็นการ sort, filter, aggregate และถึงแม้ว่าการแปลงหลายแบบจะยังไม่ถูกระบุเป็น method ใน DataFrames แต่มีให้ใช้งานใน pyspark.sql.functions [DataBricks PySpark SQL Functions](https://docs.databricks.com/aws/en/pyspark/reference/functions/) 

### Column operations

- Select
- Create
- Rename
- Cast column types
- Remove

In [0]:
# all columns in DF
df_customer.columns

#### Select columns

มีหลากหลายวิธีมากในการเลือกคอลัมน์

In [0]:
# ใช้ select and col
from pyspark.sql.functions import col

df_customer.select(
  col("c_custkey"),
  col("c_acctbal")
)

In [0]:
# ใช้ expr ที่สามารถรับ expression ในรูปแบบ stringได้
from pyspark.sql.functions import expr

df_customer.select(
  expr("c_custkey"),
  expr("c_acctbal")
)

In [0]:
# selectExpr ที่สามารถรับ SQL expressions ได้
df_customer.selectExpr(
  "c_custkey as key",
  "round(c_acctbal) as account_rounded"
)

In [0]:
# select แบบใช้ string
df_customer.select(
  "c_custkey",
  "c_acctbal"
)

ในการเลือกคอลัมน์จาก DataFrame ที่เราต้องการโดยเฉพาะ สามารถใช้ `[]` หรือ `.` ได้ ( `.` ไม่สามารถใช้เลือกคอลัมน์ที่ขึ้นต้นด้วยจำนวนเต็ม หรือคอลัมน์ที่มีช่องว่างหรืออักษรพิเศษได้) วิธีนี้ช่วยได้เมื่อ join dataframe ที่มีชื่อคอลัมน์เหมือนกัน

In [0]:
df_customer.select(
  df_customer["c_custkey"],
  df_customer["c_acctbal"]
)

In [0]:
df_customer.select(
  df_customer.c_custkey,
  df_customer.c_acctbal
)

In [0]:
# display zone
display(#your code
        )

#### Create columns

- ใช้ `withColumn`

โดยจากตัวอย่าง คือการสร้าง column ที่เก็บค่า boolean โดยค่านี้มาจากการเช็คว่า ที่ column `c_acctbal` มีค่ามากกว่า 1000 หรือไม่

In [0]:
df_customer_flag = df_customer.withColumn("balance_flag", col("c_acctbal") > 1000)

In [0]:
display(df_customer_flag)

#### Rename columns

- ใช้ `withColumnRenamed(ชื่อเดิม, ชื่อใหม่)`

In [0]:
df_customer_flag_renamed = df_customer_flag.withColumnRenamed("balance_flag", "balance_flag_renamed")

In [0]:
display(df_customer_flag_renamed)

- ใช้ `alias` เมื่อต้องการเปลี่ยนชื่อของคอลัมน์ที่มาจากการทำ aggregate

In [0]:
from pyspark.sql.functions import avg

df_segment_balance = df_customer.groupBy("c_mktsegment").agg(
    avg(df_customer["c_acctbal"]).alias("avg_account_balance")
)

display(df_segment_balance)

#### Cast column types

เปลี่ยน data type ตั้งแต่ 1+ คอลัมน์ ให้ใช้ `cast` ร่วมกันกับ `col`

In [0]:
from pyspark.sql.functions import col

df_casted = df_customer.withColumn("c_custkey", col("c_custkey").cast(StringType()))
print(type(df_casted))

#### Remove columns

สามารถละเว้นคอลัมน์ที่ไม่อยากแสดงได้ด้วยการใช้ `select * except` หรือใช้ `drop`

In [0]:
df_customer_flag_renamed.drop("balance_flag_renamed")

In [0]:
df_customer_flag_renamed.drop("c_phone", "balance_flag_renamed")

### Row operations

#### Filter rows

- ใช้ `filter` หรือ `where` เพื่อเลือกกรองเฉพาะ rows ที่เราต้องการ และถ้าต้องการระบุคอลัมน์ด้วย ให้ใช้ `col` หรือ expression ที่ประมวลผลแบบคอลัมน์

In [0]:
from pyspark.sql.functions import col

df_that_one_customer = df_customer.filter(col("c_custkey") == 412449)

- ใช้ `&` และ `|` สำหรับ `AND` และ `OR` ตามลำดับ

จากตัวอย่าง เราจะหาแถวที่ c_nationkey = 20 และ c_acctbal มากกว่า 1000

In [0]:
display(df_customer.filter((col("c_nationkey") == 20) & (col("c_acctbal") > 1000)))

In [0]:
#  ใช้ c_custkey และ or operator ร่วมกันเพื่อเลือกมาดู
df_filtered_customer = df_customer.filter((col("c_custkey") == 412446) | (col("c_custkey") == 412447))

In [0]:
display(df_filtered_customer)

#### Remove duplicate rows

In [0]:
# คัดแค่ row ที่ unique ลบแถวที่ซ้ำกันใช้คำสั่ง distinct ได้เลย
df_unique = df_customer.distinct()

#### Handle null values

ระวังสับสนระหว่าง `any` and `all`

- แบบแรก ตามโค้ดข้างล่างนี้ 2 ตัวอย่างนี้ทำงานเหมือนกัน คือ การ **drop แถวที่คอลัมน์ใดคอลัมน์หนึ่งเป็น null ออก**

In [0]:
df_customer_no_nulls = df_customer.na.drop()
df_customer_no_nulls = df_customer.na.drop("any")

- แบบที่ 2 ตามโค้ดตัวอย่างนี้ คือ การ **drop แถวที่ทุกคอลัมน์เป็น null ออก** หมายความว่า ถ้าคอลัมน์บางส่วนหายไป แถวนั้นจะยังอยู่เพราะไม่ได้ null ทั้งแถว

In [0]:
df_customer_no_nulls = df_customer.na.drop("all")

- หรือจะใช้แบบ `subset` อย่างตัวอย่างข้างล่างนี้ก็ได้ จากตัวอย่าง คือ ถ้าในคอลัมน์ที่อยู่ใน subset เป็น null ถึงจะลบแถวนั้นออก

In [0]:
df_customer_no_nulls = df_customer.na.drop("all", subset=["c_acctbal", "c_custkey"])

ในขณะเดียวกัน สามารถเติมข้อมูลที่หายไปด้วย `fill` method ได้

In [0]:
df_customer_filled = df_customer.na.fill("0", subset=["c_acctbal"])

นอกจากนี้ ยังสามารถ `replace` ข้อมูลที่เป็น **empty string** ได้ (อย่าสับสนกับ Null)

In [0]:
df_customer_phone_filled = df_customer.na.replace([""], ["UNKNOWN"], subset=["c_phone"])

#### Append rows

จาก `df_that_one_customer`ที่เราสรางไว้ตั้งแต่ตอนนี้น ตอนนี้เราจะเอามันมาเพิ่มแถวข้อมูลเข้าไปรวมกันกับ `df_filtered_customer` โดยใช้ `union`

In [0]:
df_appended_rows = df_that_one_customer.union(df_filtered_customer)

display(df_appended_rows)

#### Sort rows

- ใช้ `sort` หรือ `orderBy`

In [0]:
# Default : Ascending Order
display(df_customer.orderBy(col("c_acctbal")))

In [0]:
# ใช้ desc() เพื่อเรียงจากมากไปน้อย
display(df_customer.sort(col("c_custkey").desc()))

In [0]:
# เรียงมากกว่า 1 คอลัมน์
df_sorted = df_customer.orderBy(col("c_acctbal").desc(), col("c_custkey").asc())
df_sorted = df_customer.sort(col("c_acctbal").desc(), col("c_custkey").asc())

In [0]:
display(df_sorted.limit(10))

### Join DataFrames

การ Join ตั้งแต่ 2 DataFrames ขึ้นไป เราจะใช้ Join โดยเลือกได้ว่าเราจะให้ join แบบไหน (`how`) รวมถึง join ด้วยคอลัมน์อะไร (`on`)
- inner : default, เก็บเฉพาะแถวที่มีการ match กับ on เท่านั้น
- left :  เก็บทุกแถวจาก DataFrame แรก และเก็บเฉพาะแถวจาก DataFrame ที่สองซึ่ง match กับ DF แรกเท่านั้น
- outer : เก็บทุกแถวแบบไม่สนใจว่ามันจะ match หรือไม่

In [0]:
df_customer = spark.table('samples.tpch.customer')
df_order = spark.table('samples.tpch.orders')

df_joined = df_order.join(
  df_customer,
  on = df_order["o_custkey"] == df_customer["c_custkey"],
  how = "inner"
)

display(df_joined)

ถ้าเงื่อนไขการ join มากกว่า 1 ให้ใช้ boolean operators เช่น `&` หรือ `|`

In [0]:
df_customer = spark.table('samples.tpch.customer')
df_order = spark.table('samples.tpch.orders')

df_complex_joined = df_order.join(
  df_customer,
  on = ((df_order["o_custkey"] == df_customer["c_custkey"]) & (df_order["o_totalprice"] > 500000)),
  how = "inner"
)

display(df_complex_joined)

### Aggregate data

ในการรวมข้อมูลใน DataFrame คล้ายกับการรวมข้อมูล GROUP BY ใน SQL ให้ใช้ `groupBy` เพื่อระบุคอลัมน์ที่จะใช้ในการจัดกลุ่ม และ `agg` เพื่อระบุวิธีในการรวมข้อมูล 

และ import aggregations ทั่วไป เช่น `avg`, `sum`, `max`, และ `min` จาก pyspark.sql.functions 

ตัวอย่างข้างล่างคือการแสดงยอดคงเหลือเฉลี่ยของลูกค้าแยกตาม market segment

In [0]:
from pyspark.sql.functions import avg

# group by one column
df_segment_balance = df_customer.groupBy("c_mktsegment").agg(
    avg(df_customer["c_acctbal"])
)

display(df_segment_balance)

In [0]:
# แยกตาม nation ด้วย
from pyspark.sql.functions import avg

# group by two columns
df_segment_nation_balance = df_customer.groupBy("c_mktsegment", "c_nationkey").agg(
    avg(df_customer["c_acctbal"])
)

display(df_segment_nation_balance)

In [0]:
# นับจำนวนแถว
df_customer.count()

### Chaining calls

เรื่องนี้จะเกี่ยวกับ [Lazy Evaluation](https://docs.databricks.com/aws/en/pyspark/#data-processing) กล่าวคือ การที่เราเขียนเป็นขั้นตอนต่างๆมาก่อน ไม่ว่าจะเป็นการ filter หรือการ aggregate ต่างๆ spark จะเตรียมตัวเป็น **Execution Plan** แล้วหลังจากนั้นค่อยทำรวดเดียวจบเมื่อเจอคำสั่ง **Action** เช่น display

= งานไวขึ้นเพราะไม่ได้คำนวณใหม่ในทุกๆครั้งที่ขึ้น method ใหม่ตามที่ระบุไว้ เนื่องจากมันทำรวดเดียวตาม method 1 2 3 4 เลยจบ

In [0]:
from pyspark.sql.functions import count

df_chained = (
    df_order.filter(col("o_orderstatus") == "F") # 1. Filter
    .groupBy(col("o_orderpriority")) # 2. Group By
    .agg(count(col("o_orderkey")).alias("n_orders")) # 3. Aggregation
    .sort(col("n_orders").desc()) # 4. Sort
)

display(df_chained)

## Visualize your DataFrame

ถ้าอยากทำนอกเหนือจากการ Viz ใน DataBricks แล้ว สามารถใช้ `.pandas_api()` ได้ [Pandas API](https://docs.databricks.com/aws/en/pandas/pandas-on-spark)

In [0]:
display(df_order)

Databricks visualization. Run in Databricks to view.

## Save your data

### Save your DataFrame as a table

- ใช้โครงสร้างนี้ `catalog-name.schema-name.table-name`
- ใช้คำสั่ง `write.saveAsTable()`

สามารถดูใน table ในหน้า catalog explorer และ save ซ้ำไม่ได้เพราะมันจะขึ้นแจ้ง error ว่ามีตารางนี้อยู่แล้ว

In [0]:
df_joined.write.saveAsTable(f"workspace.default.customer_info")

### Write your DataFrame as CSV

สามารถ save ในรูปแบบ CSV file ได้ โดยใช้ `write.CSV`
และมี mode ให้เลือก 3 modes คือ
- `overwrite` คำสั่งนี้จะเขียนทับข้อมูลที่มีอยู่ทั้งหมดใน path ที่กำหนดด้วย DataFrame ที่ต้องการจะ save ปัจจุบัน
- `append` เพิ่มเนื้อหาของ DataFrame ต่อท้ายข้อมูลใน path ที่กำหนด
- `ignore` จะ fails แบบเงียบๆถ้ามีข้อมูลอยู่ใน path ที่ระบุไว้อยู่แล้ว

In [0]:
# Assign this variable your file path
file_path = "/Volumes/workspace/default/tutorial/cust_info/"

(df_joined.write
  .format("csv")
  .mode("overwrite")
  .save(file_path)
)

ปกติการ save csv แบบโค้ดข้างบน ถ้าไฟล์มีขนาดใหญ่ จะมีการแบ่งเก็บเป็นไฟล์ย่อยๆแยกออกไปตามความเหมาะสมที่ระบบจะจัดให้ ใน path ที่เรากำหนด (ดูผลลัพธ์ได้จากใน volume หรือ path ที่เรา save ไว้ หรือลองดูภาพตัวอย่างหลังจาก save แล้วใน doc) หากต้องการ save แบบไฟล์เดียว ไม่แบ่ง (แต่ไฟล์จะใหญ่มาก) ให้ลองใช้คำสั่งด้านล่าง

- `.coalesce(1)` : ลด partition ลง เหลือเพียง 1 ไฟล์ 
-  `.option("header", "true")` : ให้ save header ด้วย

In [0]:
(df_joined
 .coalesce(1)
 .write
 .option("header", "true")
 .mode("overwrite")
 .csv("/Volumes/workspace/default/tutorial/cust_info_csv_ver")
)